In [ ]:
# IP adress of this computer.
HOST = "192.168.1.100"
# Port on which the server will listen.
PORT = 1300
# Whether to preview the camera pictures to adjust parameters.
SHOW_VIDEO = True
# ID of camera to use.
CAMERA_ID = 0

In [ ]:
import json
import socketserver
import os
import base64
from io import BytesIO
from PIL import Image
import cv2
import numpy as np

In [ ]:
cam = cv2.VideoCapture(CAMERA_ID)

cv2.namedWindow("Preview")
cv2.namedWindow("Mask")

_, image = cam.read()
size = image.shape

threshold = [0, 0, 0]
color_picked = np.zeros((3))

# This mask will be used to paint everything but the screen black.
mask = np.ones(image.shape)

def on_change_threshold_h(value):
    threshold[0] = value

def on_change_threshold_s(value):
    threshold[1] = value

def on_change_threshold_v(value):
    threshold[2] = value

# Add sliders to window. This ways the user can adjust thresholds.
cv2.createTrackbar("threshold_h", "Preview", 0, 255, on_change_threshold_h)
cv2.createTrackbar("threshold_s", "Preview", 0, 255, on_change_threshold_s)
cv2.createTrackbar("threshold_v", "Preview", 0, 255, on_change_threshold_v)

def pickColor(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        _, image = cam.read()

        global color_picked
        color_picked = cv2.cvtColor(np.uint8([[image[y, x]]]), cv2.COLOR_BGR2HSV)[0][0]

# Add callback to window. This way the user can pick a color to filter for.
cv2.setMouseCallback("Preview", pickColor)

In [ ]:
while(SHOW_VIDEO):
    _, image = cam.read()
    
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    global color_picked
    color_bound1 = color_picked - threshold
    color_bound2 = color_picked + threshold
    mask = cv2.inRange(image_hsv, color_bound1, color_bound2)
    
    cv2.imshow("Preview", image)
    # Display the outcome of the currently selected background masks in second window.
    cv2.imshow("Mask", mask)
    
    # Finish loop when user presses q.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

In [ ]:
cv2.namedWindow("Result")
    
start_left = 0
end_left = size[1]
start_top = 0
end_top = size[0]

def pickCorners(event, x, y, flags, param):
    
    global start_left
    global end_left
    global start_top
    global end_top
        
    if event == cv2.EVENT_LBUTTONDOWN:
        start_left = x
        start_top = y
        
    elif event == cv2.EVENT_RBUTTONDOWN:
        end_left = x + start_left
        end_top = y + start_top
        
# Set callback for cropping of image by using left and right mouse click
# for upper left and lower right corner.
cv2.setMouseCallback("Result", pickCorners)

In [ ]:
cv2.namedWindow("Result")
threshold = 120

def on_change_threshold(value):
    global threshold
    threshold = value

# Add slider to window. This way the user can select the threshold
# for generation of binary black and white image for masks.
cv2.createTrackbar("threshold", "Result", 0, 255, on_change_threshold)
        
while(SHOW_VIDEO):
    _, inv_image = cam.read()

    inv_image = inv_image[start_top:end_top, start_left:end_left]
    inv_masked_image = cv2.bitwise_and(inv_image, inv_image, mask=mask[start_top:end_top, start_left:end_left])
    masked_image = cv2.cvtColor(np.invert(inv_masked_image), cv2.COLOR_BGR2GRAY)
    masked_image = cv2.threshold(masked_image,threshold,255,cv2.THRESH_BINARY)[1]
    
    # Show the binary result using the currently chosen threshold.
    cv2.imshow("Result", masked_image)
    
    # Finish loop when user presses q.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

In [ ]:
class TCPHandler(socketserver.StreamRequestHandler):

    def handle(self):
        print("client connected.")
        try:
            _, raw_image = cam.read()
            raw_image = raw_image[start_top:end_top, start_left:end_left]
            inv_masked_image = cv2.bitwise_and(raw_image, raw_image, mask=mask[start_top:end_top, start_left:end_left])
            header = json.loads(self.rfile.readline().strip())["header"]
            if header == "image":
                _, image_arr = cv2.imencode('.jpg', inv_masked_image)
                image_bytes = image_arr.tobytes()
                image_b64 = base64.b64encode(image_bytes)
                self.wfile.write(image_b64)
            elif header ==  "raw":
                _, image_arr = cv2.imencode('.jpg', raw_image)
                image_bytes = image_arr.tobytes()
                image_b64 = base64.b64encode(image_bytes)
                self.wfile.write(image_b64)
            else:
                masked_image = cv2.cvtColor(np.invert(inv_masked_image), cv2.COLOR_BGR2GRAY)
                masked_image = cv2.threshold(masked_image,threshold,255,cv2.THRESH_BINARY)[1]
                _, image_arr = cv2.imencode('.jpg', masked_image)
                image_bytes = image_arr.tobytes()
                image_b64 = base64.b64encode(image_bytes)
                
                self.wfile.write(image_b64)
        except Exception as e:
            print(e.message, e.args)

with socketserver.TCPServer((HOST, PORT), TCPHandler) as server:
    print("Starting server.")
    server.serve_forever()